## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-06-14-48-06 +0000,bbc,Why is UK inflation still high?,https://www.bbc.com/news/articles/c17rgd8e9gjo...
1,2025-11-06-14-45-04 +0000,nyt,Former House Speaker Nancy Pelosi Will Retire ...,https://www.nytimes.com/2025/11/06/us/politics...
2,2025-11-06-14-43-59 +0000,wapo,Body of American Israeli soldier Itay Chen amo...,https://www.washingtonpost.com/world/2025/10/1...
3,2025-11-06-14-42-00 +0000,wsj,U.S. to Reduce Flight Traffic by 10% at 40 Air...,https://www.wsj.com/business/airlines/u-s-to-r...
4,2025-11-06-14-42-00 +0000,wsj,The FAA said it was ordering airline traffic t...,https://www.wsj.com/business/airlines/u-s-to-r...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,trump,53
328,mamdani,33
258,new,26
178,election,20
327,zohran,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
160,2025-11-06-05-58-57 +0000,nypost,Florida sheriff invites New Yorkers looking to...,https://nypost.com/2025/11/06/us-news/florida-...,164
220,2025-11-05-23-44-26 +0000,nyt,Trump vs. Mamdani: President Sees a Capable Fo...,https://www.nytimes.com/2025/11/05/us/politics...,147
299,2025-11-05-20-26-30 +0000,nypost,New York Post’s cover on Zohran Mamdani electi...,https://nypost.com/2025/11/05/us-news/new-york...,138
305,2025-11-05-20-10-00 +0000,wsj,Zohran Mamdani sealed his mayoral victory in N...,https://www.wsj.com/politics/elections/the-vot...,134
191,2025-11-06-02-00-00 +0000,wsj,Economic discontent helped deliver election wi...,https://www.wsj.com/politics/elections/economi...,130


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
160,164,2025-11-06-05-58-57 +0000,nypost,Florida sheriff invites New Yorkers looking to...,https://nypost.com/2025/11/06/us-news/florida-...
391,117,2025-11-05-15-00-06 +0000,nyt,A Supreme Court Veteran and a Newcomer Will Ma...,https://www.nytimes.com/2025/11/05/us/politics...
28,98,2025-11-06-14-00-00 +0000,wsj,Former House Speaker Nancy Pelosi says she won...,https://www.wsj.com/politics/former-house-spea...
4,78,2025-11-06-14-42-00 +0000,wsj,The FAA said it was ordering airline traffic t...,https://www.wsj.com/business/airlines/u-s-to-r...
257,60,2025-11-05-22-07-00 +0000,wsj,"Maine Rep. Jared Golden, one of the most vulne...",https://www.wsj.com/politics/elections/jared-g...
38,58,2025-11-06-13-42-00 +0000,wsj,Employers said they cut more than a million jo...,https://www.wsj.com/economy/jobs/planned-u-s-j...
43,47,2025-11-06-13-23-00 +0000,wsj,The Bank of England left its key interest rate...,https://www.wsj.com/economy/central-banking/ba...
239,43,2025-11-05-22-53-33 +0000,nypost,‘Tutors’ at Musk startup xAI had to give up ri...,https://nypost.com/2025/11/05/business/tutors-...
299,42,2025-11-05-20-26-30 +0000,nypost,New York Post’s cover on Zohran Mamdani electi...,https://nypost.com/2025/11/05/us-news/new-york...
83,40,2025-11-06-11-17-29 +0000,cbc,Vietnam evacuates hundreds of thousands ahead ...,https://www.cbc.ca/news/world/typhoon-kalmaegi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
